In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import geopandas as gpd
import json

import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings('ignore')

In [2]:
DATA_DIR = Path("./../data")
RAW_FILE = DATA_DIR / "raw/Sales_of_real_estate_N_per_quarter_per_municipality_2010_2025.xlsx"
PROCESSED_FILE = DATA_DIR / "processed/Sales_of_real_estate_N_per_quarter_per_municipality_2010_2025_processed.xlsx"

In [3]:
df = pd.read_excel(PROCESSED_FILE, sheet_name="Par commune", header=0)
df.head()

,refnis,localite,annee,periode,nombre_transactions,variable,value,property_type,metric,quarter_str,date
0,11001,AARTSELAAR,2010,Q1,29.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",252000.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",prix_médian(€),2010Q1,2010-03-31
1,11001,AARTSELAAR,2010,Q2,25.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",254000.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",prix_médian(€),2010Q2,2010-06-30
2,11001,AARTSELAAR,2010,Q3,21.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",255000.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",prix_médian(€),2010Q3,2010-09-30
3,11001,AARTSELAAR,2010,Q4,28.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",245000.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",prix_médian(€),2010Q4,2010-12-31
4,11001,AARTSELAAR,2011,Q1,23.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",310000.0,"toutes_les_maisons_avec_2,_3,_4_ou_plus_de_faç...",prix_médian(€),2011Q1,2011-03-31


# base

In [4]:
# Keep only relevant price metrics
price_index = (
    df[df["metric"].isin(["prix_médian(€)", "prix_premier_quartile(€)", "prix_troisième_quartile(€)"])]
    .pivot_table(
        index=["refnis", "property_type", "date"],
        columns="metric",
        values="value",
        aggfunc="median"
    )
    .reset_index()
    .rename(columns={
        "prix_médian(€)": "median_price",
        "prix_premier_quartile(€)": "q1_price",
        "prix_troisième_quartile(€)": "q3_price",
    })
)

In [5]:
# Interpolate for each municipality + property_type
def interpolate_prices(group):
    group = group.set_index("date").sort_index()
    group = group.resample("QE").mean()
    for col in ["median_price", "q1_price", "q3_price"]:
        group[col] = group[col].interpolate(method="linear")
    return group.reset_index()

price_index_interp = (
    price_index.groupby(["refnis", "property_type"], group_keys=True)
    .apply(interpolate_prices)
    .reset_index()
    .drop(columns=["level_2"])
)

## What does following code do
This code adjusts historical property prices to account for market changes over time. Think of it like adjusting for inflation, but specifically for real estate prices in a particular area.
The main function, price_corrector_exact(), answers this question: "If a house was listed for €250,000 in 2010, what would that be worth in 2020 based on how the local market changed?"
## How It Works
The function takes a property's original listing price and adjusts it based on how prices moved in that specific municipality and property type between two dates. It's strict about dates though - you can only use exact quarter-end dates that exist in the dataset (like March 31, June 30, etc.). If you try to use a random date, it'll tell you which dates are actually available.
The calculation is straightforward: it finds the price index for your starting date (t0) and ending date (t1), then scales your listing price by the ratio between them. So if the market went up 50% during that period, your €250,000 listing would be corrected to €375,000.
The Example
The code demonstrates this by taking a €250,000 listing from March 31, 2010 and adjusting it to June 30, 2020 for a specific property type in municipality 11001 (which is Antwerp). It does this three times using different price metrics:

- Median price - the typical middle-of-the-market property
- Q1 price - lower-end properties (first quartile)
- Q3 price - higher-end properties (third quartile)

This shows you how different segments of the market might have changed at different rates. Budget homes might have appreciated differently than luxury properties, and this lets you see all three perspectives.

In [6]:
def price_corrector_exact(listing_price, refnis, property_type, t0, t1, metric="median_price"):
    """
    Corrects listing_price from time t0 to t1 for a given municipality and property type.
    Works only with dates available in the dataframe (no interpolation).
    """
    subset = price_index_interp.query("refnis == @refnis and property_type == @property_type")

    if subset.empty:
        raise ValueError("No price data for given refnis/property_type.")

    # Make sure date is datetime
    t0 = pd.to_datetime(t0)
    t1 = pd.to_datetime(t1)

    # Restrict to available dates only
    available_dates = subset["date"].unique()

    if t0 not in available_dates or t1 not in available_dates:
        raise ValueError(
            f"Dates must be exact quarter-end dates available in the dataset.\n"
            f"Available examples: {sorted(available_dates)[:5]} ... {sorted(available_dates)[-5:]}"
        )

    # Get exact values
    p0 = subset.loc[subset["date"] == t0, metric].values[0]
    p1 = subset.loc[subset["date"] == t1, metric].values[0]

    if pd.isna(p0) or pd.isna(p1):
        raise ValueError("Price data unavailable for one of the dates.")

    corrected_price = listing_price * (p1 / p0)
    return corrected_price


corrected_median = price_corrector_exact(
    listing_price=250_000,
    refnis=11001,
    property_type="toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)",
    t0="2010-03-31",  # must be an available quarter end
    t1="2020-06-30",  # must be an available quarter end
    metric="median_price"
)

corrected_q1 = price_corrector_exact(
    listing_price=250_000,
    refnis=11001,
    property_type="toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)",
    t0="2010-03-31",  # must be an available quarter end
    t1="2020-06-30",  # must be an available quarter end
    metric="q1_price"
)


corrected_q3 = price_corrector_exact(
    listing_price=250_000,
    refnis=11001,
    property_type="toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)",
    t0="2010-03-31",  # must be an available quarter end
    t1="2020-06-30",  # must be an available quarter end
    metric="q3_price"
)

print(f"Corrected (Median): €{corrected_median:,.0f}")
print(f"Corrected (Q1): €{corrected_q1:,.0f}")
print(f"Corrected (Q3): €{corrected_q3:,.0f}")

Corrected (Median): €314,484
Corrected (Q1): €316,667
Corrected (Q3): €331,897


## What Following Version Does Differently
This is a more flexible version of the price correction function. While it does the same basic job - adjusting a historical property price based on market changes - it removes the strict date requirement from the earlier version.

## The Key Difference: Smart Date Handling

Instead of demanding exact quarter-end dates, this version uses a clever technique called .asof(). Think of it like this: if you ask "what was the price on May 10, 2010?" and that exact date isn't in the dataset, it will find the most recent available data before that date and use that instead.
So when you use dates like May 10, 2010 or July 15, 2020 (which aren't quarter-ends), the function essentially says "okay, I'll use the closest quarterly data I have before those dates." This makes it much more user-friendly - you can use any date, even listing dates or transaction dates that don't line up perfectly with when the data was collected.

## Same Goal, Easier to Use

The example still corrects a €250,000 property from 2010 to 2020 for the same Antwerp property type, and it still calculates three versions (median, Q1, and Q3 prices). The calculation logic is identical - it's just that now you don't have to look up which exact dates are available in the dataset first.
This is the version you'd typically want to use in practice, since real estate transactions and listings happen on arbitrary dates, not just on convenient quarter-ends.

In [7]:
def price_corrector(listing_price, refnis, property_type, t0, t1, metric="median_price"):
    """
    Corrects listing_price from time t0 to t1 for a given municipality and property type.
    Allows metric = 'median_price', 'q1_price', or 'q3_price'.
    """
    subset = price_index_interp.query("refnis == @refnis and property_type == @property_type")

    if subset.empty:
        raise ValueError("No price data for given refnis/property_type.")

    subset = subset.set_index("date")

    p0 = subset[metric].asof(pd.to_datetime(t0))
    p1 = subset[metric].asof(pd.to_datetime(t1))

    if pd.isna(p0) or pd.isna(p1):
        raise ValueError("Price data unavailable for one of the dates.")

    corrected_price = listing_price * (p1 / p0)
    return corrected_price



corrected_median = price_corrector(
    listing_price=250_000,
    refnis=11001,
    property_type="toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)",
    t0="2010-05-10",
    t1="2020-07-15",
    metric="median_price"
)


corrected_q1 = price_corrector(
    listing_price=250_000,
    refnis=11001,
    property_type="toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)",
    t0="2010-05-10",
    t1="2020-07-15",
    metric="q1_price"
)


corrected_q3 = price_corrector(
    listing_price=250_000,
    refnis=11001,
    property_type="toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)",
    t0="2010-05-10",
    t1="2020-07-15",
    metric="q3_price"
)

print(f"Corrected (Median): €{corrected_median:,.0f}")
print(f"Corrected (Q1): €{corrected_q1:,.0f}")
print(f"Corrected (Q3): €{corrected_q3:,.0f}")

Corrected (Median): €314,484
Corrected (Q1): €316,667
Corrected (Q3): €331,897


# forcast

## What This Code Does

This code predicts future property prices using statistical time series models. It's like looking at how prices have moved historically and using that pattern to make educated guesses about where they're headed next.

## forecast_prices() - The Core Function

This is the engine that does the actual forecasting. It takes your historical price data and projects it forward for however many quarters you want (default is 4 quarters = 1 year ahead).

It can use two different forecasting methods:
1. **ARIMA:** A statistical model that looks at trends and patterns in how prices change over time. It first checks if your data is "stationary" (meaning it fluctuates around a stable average) and adjusts its approach accordingly. Think of it as identifying whether prices are steadily climbing, falling, or bouncing around, then projecting that pattern forward.
2. **Exponential Smoothing (Holt-Winters):** This method gives more weight to recent data and can capture seasonal patterns (like if prices typically rise in spring). If you have enough data (at least 2 years of quarterly data), it'll look for these seasonal cycles.

The function can run both methods and compare them, or you can pick just one. It also checks that you have enough historical data to make meaningful predictions (at least 8 quarters).

## compare_forecasts() - See Both Predictions Side by Side

This function runs both ARIMA and Exponential Smoothing, then shows you their predictions together. It's helpful because different models can give slightly different forecasts, and seeing both gives you a range of possibilities. It even calculates an average of the two forecasts as a middle-ground estimate.
The output shows you model statistics (AIC scores - lower is better) and the actual forecasted prices for each future quarter.

## forecast_and_correct() - Bringing It All Together

This is where forecasting meets the earlier price correction concept. It answers questions like: "I have a property listed at €250,000 today - what would it be worth in 1 or 2 years based on predicted market trends?"
It takes your listing price from a historical date, forecasts where the market is heading, and shows you what that listing would be worth at various future dates. It also shows the percentage change, so you can see if the market is expected to grow 5%, 10%, 15%, etc.

## Practical Use

Unlike the previous functions that corrected prices between two known historical dates, this code lets you peek into the future. It's particularly useful for investment decisions, long-term planning, or understanding where a market might be heading. Of course, these are predictions based on past patterns, not guarantees - real estate markets can be influenced by factors the model can't see coming.

In [8]:
def forecast_prices(refnis, property_type, metric="median_price", periods=4, method="auto"):
    """
    Forecasts future prices for a given municipality and property type.
    
    Parameters:
    refnis : int
        Municipality reference number
    property_type : str
        Type of property
    metric : str
        Price metric to forecast ('median_price', 'q1_price', 'q3_price')
    periods : int
        Number of quarters to forecast ahead
    method : str
        Forecasting method: 'arima', 'exp_smoothing', or 'auto' (tries both)
    
    Returns:
    dict with forecasted values and model info
    """
    # Filter and prepare data
    subset = price_index_interp.query("refnis == @refnis and property_type == @property_type").copy()
    if subset.empty:
        raise ValueError("No price data for given refnis/property_type.")
    
    subset = subset.set_index("date").sort_index()
    ts = subset[metric].dropna()
    
    if len(ts) < 8:
        raise ValueError("Insufficient data points for forecasting (need at least 8).")
    
    results = {}
    
    # Method 1: ARIMA
    if method in ["arima", "auto"]:
        try:
            # Check stationarity
            adf_result = adfuller(ts)
            is_stationary = adf_result[1] < 0.05
            
            # Auto-select order or use simple default
            # For price data, typically use (1,1,1) or (1,0,1)
            d = 0 if is_stationary else 1
            
            model_arima = ARIMA(ts, order=(1, d, 1))
            fitted_arima = model_arima.fit()
            
            # Forecast
            forecast_arima = fitted_arima.forecast(steps=periods)
            
            # Generate future dates (quarterly)
            last_date = ts.index[-1]
            future_dates = pd.date_range(
                start=last_date + pd.DateOffset(months=3),
                periods=periods,
                freq='QS'
            )
            
            results['arima'] = {
                'forecast': pd.Series(forecast_arima.values, index=future_dates),
                'model': fitted_arima,
                'aic': fitted_arima.aic,
                'bic': fitted_arima.bic,
                'order': (1, d, 1),
                'is_stationary': is_stationary
            }
        except Exception as e:
            results['arima'] = {'error': str(e)}
    
    # Method 2: Exponential Smoothing (Holt-Winters)
    if method in ["exp_smoothing", "auto"]:
        try:
            # Try with seasonal component if enough data
            seasonal_periods = 4  # Quarterly data
            use_seasonal = len(ts) >= 2 * seasonal_periods
            
            if use_seasonal:
                model_es = ExponentialSmoothing(
                    ts,
                    trend='add',
                    seasonal='add',
                    seasonal_periods=seasonal_periods
                )
            else:
                model_es = ExponentialSmoothing(ts, trend='add', seasonal=None)
            
            fitted_es = model_es.fit()
            
            # Forecast
            forecast_es = fitted_es.forecast(steps=periods)
            
            # Generate future dates
            last_date = ts.index[-1]
            future_dates = pd.date_range(
                start=last_date + pd.DateOffset(months=3),
                periods=periods,
                freq='QS'
            )
            
            results['exp_smoothing'] = {
                'forecast': pd.Series(forecast_es.values, index=future_dates),
                'model': fitted_es,
                'aic': fitted_es.aic,
                'seasonal': use_seasonal
            }
        except Exception as e:
            results['exp_smoothing'] = {'error': str(e)}
    
    # Add original time series for reference
    results['historical'] = ts
    results['last_date'] = ts.index[-1]
    results['last_value'] = ts.iloc[-1]
    
    return results


def compare_forecasts(refnis, property_type, metric="median_price", periods=4):
    """
    Compare ARIMA and Exponential Smoothing forecasts side by side.
    """
    results = forecast_prices(refnis, property_type, metric, periods, method="auto")
    
    print(f"\n{'='*80}")
    print(f"FORECAST COMPARISON")
    print(f"Municipality: {refnis} | Property Type: {property_type}")
    print(f"Metric: {metric}")
    print(f"{'='*80}\n")
    
    print(f"Historical Data: {len(results['historical'])} quarters")
    print(f"Last Known Value ({results['last_date'].date()}): €{results['last_value']:,.0f}")
    print(f"\nForecasting {periods} quarters ahead:\n")
    
    # Create comparison DataFrame
    forecast_df = pd.DataFrame()
    
    if 'arima' in results and 'forecast' in results['arima']:
        forecast_df['ARIMA'] = results['arima']['forecast']
        print(f"ARIMA Model (Order: {results['arima']['order']}):")
        print(f"  AIC: {results['arima']['aic']:.2f}")
        print(f"  BIC: {results['arima']['bic']:.2f}")
        print()
    
    if 'exp_smoothing' in results and 'forecast' in results['exp_smoothing']:
        forecast_df['Exp_Smoothing'] = results['exp_smoothing']['forecast']
        seasonal_text = "with" if results['exp_smoothing']['seasonal'] else "without"
        print(f"Exponential Smoothing ({seasonal_text} seasonality):")
        print(f"  AIC: {results['exp_smoothing']['aic']:.2f}")
        print()
    
    if not forecast_df.empty:
        # Calculate average if both available
        if len(forecast_df.columns) == 2:
            forecast_df['Average'] = forecast_df.mean(axis=1)
        
        print("\nForecasted Prices:")
        print("-" * 80)
        for idx, row in forecast_df.iterrows():
            print(f"{idx.date()}:")
            for col in forecast_df.columns:
                print(f"  {col:20s}: €{row[col]:,.0f}")
            print()
    
    return results, forecast_df


def forecast_and_correct(listing_price, refnis, property_type, t0, future_quarters=4, metric="median_price"):
    """
    Forecast future prices and correct a listing price to various future dates.
    """
    results = forecast_prices(refnis, property_type, metric, periods=future_quarters, method="auto")
    
    print(f"\n{'='*80}")
    print(f"PRICE CORRECTION WITH FORECASTING")
    print(f"{'='*80}\n")
    print(f"Original Listing Price: €{listing_price:,.0f}")
    print(f"Original Date: {t0}")
    print(f"Municipality: {refnis}")
    print()
    
    # Get historical price at t0
    subset = price_index_interp.query("refnis == @refnis and property_type == @property_type")
    subset = subset.set_index("date")
    p0 = subset[metric].asof(pd.to_datetime(t0))
    
    if pd.isna(p0):
        raise ValueError(f"No price data available for {t0}")
    
    print(f"Base Price Index at {t0}: €{p0:,.0f}\n")
    
    # Use ARIMA forecasts (preferred for trend-based data)
    if 'arima' in results and 'forecast' in results['arima']:
        forecasts = results['arima']['forecast']
        
        print("Corrected Prices for Future Quarters (using ARIMA):")
        print("-" * 80)
        
        for date, p1 in forecasts.items():
            corrected = listing_price * (p1 / p0)
            change_pct = ((p1 / p0) - 1) * 100
            print(f"{date.date()}: €{corrected:,.0f}  (change: {change_pct:+.1f}%)")
    
    return results

In [9]:
print("EXAMPLE 1: Comparing Forecasting Methods")
print("="*80)

results1, forecast_df1 = compare_forecasts(
    refnis=11001,
    property_type="toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)",
    metric="median_price",
    periods=4
)

EXAMPLE 1: Comparing Forecasting Methods

FORECAST COMPARISON
Municipality: 11001 | Property Type: toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)
Metric: median_price

Historical Data: 62 quarters
Last Known Value (2025-06-30): €425,000

Forecasting 4 quarters ahead:

ARIMA Model (Order: (1, 1, 1)):
  AIC: 1430.39
  BIC: 1436.73

Exponential Smoothing (with seasonality):
  AIC: 1267.12


Forecasted Prices:
--------------------------------------------------------------------------------
2025-10-01:
  ARIMA               : €426,899
  Exp_Smoothing       : €434,406
  Average             : €430,653

2026-01-01:
  ARIMA               : €427,132
  Exp_Smoothing       : €435,853
  Average             : €431,493

2026-04-01:
  ARIMA               : €427,161
  Exp_Smoothing       : €433,436
  Average             : €430,299

2026-07-01:
  ARIMA               : €427,165
  Exp_Smoothing       : €447,131
  Average             : €437,148



In [10]:
print("EXAMPLE 2: Correcting Listing Price with Forecast")
print("="*80)

results2 = forecast_and_correct(
    listing_price=250_000,
    refnis=11001,
    property_type="toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)",
    t0="2010-05-10",
    future_quarters=6,
    metric="median_price"
)

EXAMPLE 2: Correcting Listing Price with Forecast

PRICE CORRECTION WITH FORECASTING

Original Listing Price: €250,000
Original Date: 2010-05-10
Municipality: 11001

Base Price Index at 2010-05-10: €252,000

Corrected Prices for Future Quarters (using ARIMA):
--------------------------------------------------------------------------------
2025-10-01: €423,511  (change: +69.4%)
2026-01-01: €423,743  (change: +69.5%)
2026-04-01: €423,771  (change: +69.5%)
2026-07-01: €423,775  (change: +69.5%)
2026-10-01: €423,775  (change: +69.5%)
2027-01-01: €423,775  (change: +69.5%)


In [11]:
print("EXAMPLE 3: Forecasting Different Price Metrics")
print("="*80)

for metric in ["q1_price", "median_price", "q3_price"]:
    print(f"\n--- {metric.upper()} ---")
    results = forecast_prices(
        refnis=11001,
        property_type="toutes_les_maisons_avec_2,_3,_4_ou_plus_de_façades_(excl._appartements)",
        metric=metric,
        periods=4,
        method="arima"
    )
    
    if 'arima' in results and 'forecast' in results['arima']:
        print(f"Last known: €{results['last_value']:,.0f}")
        print("Forecasts:")
        for date, price in results['arima']['forecast'].items():
            print(f"  {date.date()}: €{price:,.0f}")

EXAMPLE 3: Forecasting Different Price Metrics

--- Q1_PRICE ---
Last known: €370,000
Forecasts:
  2025-10-01: €367,390
  2026-01-01: €367,200
  2026-04-01: €367,186
  2026-07-01: €367,185

--- MEDIAN_PRICE ---
Last known: €425,000
Forecasts:
  2025-10-01: €426,899
  2026-01-01: €427,132
  2026-04-01: €427,161
  2026-07-01: €427,165

--- Q3_PRICE ---
Last known: €472,500
Forecasts:
  2025-10-01: €513,961
  2026-01-01: €516,560
  2026-04-01: €516,723
  2026-07-01: €516,733


# Other Forecasting Approaches Considered

While working on this forecasting problem, I evaluated several other methods before settling on the ARIMA and Exponential Smoothing approach:

## Linear Trend Extrapolation

The simplest option would be fitting a straight line through historical prices and extending it forward. This is very easy to understand and implement. 
If prices have been going up €10,000 per year, just keep adding €10,000. 
However, real estate markets rarely grow in perfectly straight lines. They tend to have periods of rapid growth, plateaus, and sometimes corrections. 
A linear model would miss all that nuance and could significantly under or overestimate future values, especially for longer-term forecasts.

## Machine Learning Models (Prophet, XGBoost, LightGBM)

On the other end of the spectrum, I could have used sophisticated machine learning models. Facebook's Prophet is particularly popular for time series forecasting and can handle complex patterns, holidays, and multiple influencing factors. 
Gradient boosting models like XGBoost could potentially capture nonlinear relationships really well.
However, They require significant tuning, more features to train on, and extensive justification for why you're using them. 
While they might squeeze out a bit more accuracy, the added complexity and implementation time didn't seem to work for a time-boxed interview task in my current state. 

## Index Growth Projection

Another approach would be to calculate the average quarterly growth rate from historical data and apply it repeatedly. 
For example, if prices grew 1.5% per quarter on average, just compound that forward. 
This has the advantage of being conceptually aligned with the price correction logic I had already built, both rely on the same underlying price index data.
The downside is that it assumes constant growth and ignores any patterns, momentum, or changing market conditions. 
It's basically a slightly more sophisticated version of the linear trend.

## Why I Chose ARIMA and Exponential Smoothing
After considering these options, I found that ARIMA and Exponential Smoothing strike the right balance for this task. They're:

- Sophisticated enough to provide reasonable results
- Not overly complex so they can be implemented reasonably quickly with standard libraries
- Appropriate for the data since they're designed specifically for time series and can capture trends and seasonality without requiring external features